# Chapter 16: Storage and Volumes

While Kubernetes excels at orchestrating stateless applications, modern systems inevitably require durable storage for databases, message queues, content repositories, and session data. Chapter 15 established the communication fabric; this chapter addresses data persistence across the container lifecycle. We distinguish between ephemeral storage (tied to Pod lifetimes) and persistent storage (surviving Pod rescheduling), examining the abstractions Kubernetes provides to decouple storage consumption from storage implementation.

Understanding PersistentVolumes, StorageClasses, and StatefulSets is essential for running production databases and stateful services in Kubernetes, moving beyond the stateless microservices patterns toward enterprise-grade data management.

## 16.1 Volume Types

Kubernetes supports multiple volume types, each suited to specific use cases ranging from temporary scratch space to high-performance persistent network storage.

### Ephemeral vs. Persistent

**Ephemeral Volumes:**
- Lifecycle bound to Pod existence
- Destroyed when Pod is deleted
- Suitable for temporary data, caching, scratch space
- Types: `emptyDir`, `configMap`, `secret`, `downwardAPI`

**Persistent Volumes:**
- Exist beyond Pod lifecycle
- Survive node failures and Pod rescheduling
- Require backend storage (cloud disks, NFS, SAN)
- Types: Implemented via PersistentVolume abstraction

### emptyDir

The simplest volume type, created when a Pod is assigned to a node and exists as long as the Pod runs on that node.

**Characteristics:**
- Initially empty
- Shared between containers in the same Pod
- Stored on node disk (or RAM with `medium: Memory`)
- Deleted when Pod leaves the node (not just restarted)

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: scratch-space
spec:
  containers:
  - name: processor
    image: busybox
    command: ['sh', '-c', 'while true; do echo $(date) >> /scratch/log.txt; sleep 5; done']
    volumeMounts:
    - name: shared-data
      mountPath: /scratch
  - name: reader
    image: busybox
    command: ['sh', '-c', 'tail -f /scratch/log.txt']
    volumeMounts:
    - name: shared-data
      mountPath: /scratch
  volumes:
  - name: shared-data
    emptyDir:
      medium: Memory  # Optional: tmpfs (RAM-backed), faster but counts against memory limit
      sizeLimit: 500Mi  # Enforced eviction when exceeded
```

**Use Cases:**
- Shared temporary workspace for multi-container Pods
- Caching layers that rebuild on restart
- Sorting/scratch space for data processing

**Caution:** `emptyDir` with `medium: Memory` consumes node RAM and is included in container memory limits. Exceeding `sizeLimit` triggers Pod eviction.

### hostPath

Mounts a file or directory from the host node's filesystem into the Pod.

**Security Warning:** hostPath exposes host filesystem, creating security risks and breaking Pod portability. Use only for:
- Accessing Docker internals (socket, cgroup)
- Node-level logging agents
- Storage drivers requiring host access
- Single-node development (Minikube)

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: node-logger
spec:
  containers:
  - name: logger
    image: fluentd
    volumeMounts:
    - name: host-logs
      mountPath: /var/log/host
      readOnly: true  # Always use readOnly when possible
  volumes:
  - name: host-logs
    hostPath:
      path: /var/log
      type: Directory  # Must exist or Pod fails to start
      # Alternatives: DirectoryOrCreate, File, FileOrCreate, Socket, CharDevice, BlockDevice
```

**Restrictions:**
- Read-only recommended to prevent container escape
- Path must exist on node (unless using `*OrCreate` types)
- Not portable across nodes with different filesystem layouts

### projected

Projects multiple volume sources into a single directory, useful for combining ConfigMaps, Secrets, and downwardAPI:

```yaml
volumes:
- name: projected-config
  projected:
    sources:
    - secret:
        name: db-credentials
        items:
        - key: username
          path: db/user.txt
    - configMap:
        name: app-config
        items:
        - key: settings.json
          path: config/settings.json
    - downwardAPI:
        items:
        - path: labels.json
          fieldRef:
            fieldPath: metadata.labels
    defaultMode: 0440
```

## 16.2 Persistent Volumes (PV)

PersistentVolumes represent storage resources provisioned by administrators or dynamic provisioners. They exist as cluster-level resources independent of individual Pods, decoupling storage lifecycle from application lifecycle.

### PV Lifecycle

**Provisioning:**
- **Static**: Administrator creates PVs in advance
- **Dynamic**: StorageClass creates PVs on demand (preferred)

**Binding:**
- PVC requests bind to available PVs matching criteria
- One-to-one relationship (PV to PVC)
- Binding is exclusive; released PVs must be reclaimed

**Using:**
- PV mounted as volume in Pod
- Specified via PVC reference in Pod spec

**Reclaiming:**
- **Retain**: Manual reclamation required (data preserved)
- **Delete**: PV and underlying storage deleted (default for dynamic provisioning)
- **Recycle**: Deprecated; basic scrub (rm -rf /thevolume/*)

### PV Specification

```yaml
apiVersion: v1
kind: PersistentVolume
metadata:
  name: pv-nfs-001
  labels:
    type: nfs
    tier: production
spec:
  capacity:
    storage: 100Gi
  volumeMode: Filesystem  # or Block for raw block devices
  accessModes:
    - ReadWriteOnce      # RWO: Single node read-write
    # - ReadOnlyMany     # ROX: Multiple nodes read-only
    # - ReadWriteMany    # RWX: Multiple nodes read-write (requires NFS, CephFS, etc.)
    # - ReadWriteOncePod # RWOP: Single Pod read-write (Kubernetes 1.22+)
  persistentVolumeReclaimPolicy: Retain
  storageClassName: nfs-slow
  mountOptions:
    - hard
    - nfsvers=4.1
  nfs:
    server: nfs.company.internal
    path: /exports/data-001
```

**Access Modes Explained:**

| Mode | Abbreviation | Use Case | Supported By |
|------|--------------|----------|--------------|
| ReadWriteOnce | RWO | Single Pod read-write (most databases) | GCE PD, EBS, iSCSI, NFS |
| ReadOnlyMany | ROX | Multiple Pods read-only (shared config) | NFS, CephFS, Azure File |
| ReadWriteMany | RWX | Multiple Pods read-write (shared storage) | NFS, CephFS, GlusterFS, Azure File |
| ReadWriteOncePod | RWOP | Single Pod exclusive access (newer, safer) | Most block storage |

**Volume Modes:**
- **Filesystem**: Mounts into directory (default)
- **Block**: Raw block device for databases needing direct I/O (bypassing filesystem overhead)

### NFS Example

Network File System provides shared storage accessible from multiple nodes:

```yaml
apiVersion: v1
kind: PersistentVolume
metadata:
  name: pv-shared-assets
spec:
  capacity:
    storage: 500Gi
  accessModes:
    - ReadWriteMany
  nfs:
    server: 10.0.0.50
    path: /var/nfs/shared-assets
  mountOptions:
    - vers=4.1
    - nconnect=16  # Multiple connections for better throughput
    - noatime      # Disable access time updates for performance
```

### Cloud Provider Examples

**AWS EBS (Block Storage):**
```yaml
apiVersion: v1
kind: PersistentVolume
metadata:
  name: pv-aws-ebs
spec:
  capacity:
    storage: 100Gi
  accessModes:
    - ReadWriteOnce
  awsElasticBlockStore:
    volumeID: vol-0a1234567890abcdef0
    fsType: ext4
    # Optional: IOPS and throughput for gp3/io1/io2
```

**GCP Persistent Disk:**
```yaml
apiVersion: v1
kind: PersistentVolume
metadata:
  name: pv-gcp-pd
spec:
  gcePersistentDisk:
    pdName: my-data-disk
    fsType: ext4
```

**Azure Disk:**
```yaml
apiVersion: v1
kind: PersistentVolume
metadata:
  name: pv-azure-disk
spec:
  azureDisk:
    diskName: mydisk.vhd
    diskURI: https://myaccount.blob.core.windows.net/vhds/mydisk.vhd
    cachingMode: ReadOnly
    fsType: ext4
    readOnly: false
```

## 16.3 Persistent Volume Claims (PVC)

PersistentVolumeClaims are user requests for storage. Developers consume storage through PVCs without knowing underlying infrastructure details, enabling portability across environments.

### PVC Specification

```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: postgres-data
  namespace: database
spec:
  accessModes:
    - ReadWriteOnce
  volumeMode: Filesystem
  resources:
    requests:
      storage: 50Gi
  storageClassName: fast-ssd  # Optional: triggers dynamic provisioning
  selector:
    matchLabels:
      tier: production
    matchExpressions:
      - {key: type, operator: In, values: [ssd, nvme]}
```

**Key Fields:**
- **accessModes**: Must match PV capabilities (subset of PV modes)
- **storageClassName**: Links to StorageClass for dynamic provisioning; empty string for static binding
- **resources.requests.storage**: Minimum capacity required
- **selector**: Optional label matching for specific PVs

### Binding Process

1. User creates PVC requesting 50Gi, RWO, StorageClass "fast-ssd"
2. Control plane searches for matching PVs or triggers StorageClass provisioner
3. If static: Binds to available PV meeting criteria
4. If dynamic: Creates new PV of requested size, then binds
5. PVC status changes to "Bound"
6. Pod references PVC by name in volume mounts

### Using PVCs in Pods

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: postgres-0
spec:
  containers:
  - name: postgres
    image: postgres:15
    volumeMounts:
    - name: data
      mountPath: /var/lib/postgresql/data
      subPath: postgres  # Mounts specific subdirectory, prevents volume root clutter
  volumes:
  - name: data
    persistentVolumeClaim:
      claimName: postgres-data
      readOnly: false
```

**subPath Usage:**
- Mounts specific subdirectory rather than volume root
- Prevents database files from mixing with lost+found or other system directories
- Enables multiple databases to share one PV (not recommended for production)

### Expanding PVCs

Kubernetes supports online volume expansion for many storage classes:

```yaml
# Edit PVC to increase size
kubectl patch pvc postgres-data -p '{"spec":{"resources":{"requests":{"storage":"100Gi"}}}}'

# Verify expansion
kubectl get pvc postgres-data -o jsonpath='{.status.capacity.storage}'
```

**Requirements:**
- StorageClass must have `allowVolumeExpansion: true`
- Cloud provider/CSI driver must support expansion
- File system may require manual expansion inside Pod (xfs_growfs, resize2fs)

## 16.4 Storage Classes

StorageClasses provide abstraction over storage types, enabling dynamic provisioning without pre-created PVs. They define "classes" of storage (fast SSD, archival, replicated, etc.) with different performance and cost characteristics.

### StorageClass Definition

```yaml
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: fast-ssd
  annotations:
    storageclass.kubernetes.io/is-default-class: "true"  # Used when PVC omits storageClassName
provisioner: kubernetes.io/gce-pd  # CSI driver or in-tree provisioner
parameters:
  type: pd-ssd  # SSD persistent disk
  replication-type: regional  # Replicated across zones
  zones: us-central1-a,us-central1-b
volumeBindingMode: WaitForFirstConsumer  # Delay provisioning until Pod scheduled
allowVolumeExpansion: true
mountOptions:
  - debug
  - noatime
reclaimPolicy: Delete  # Delete underlying disk when PVC deleted
```

**Common Provisioners:**

| Provisioner | Platform | Parameters |
|-------------|----------|------------|
| kubernetes.io/aws-ebs | AWS | type (gp2/gp3/io1/io2), iops, encrypted, kmsKeyId |
| kubernetes.io/gce-pd | GCP | type (pd-standard/pd-ssd/pd-balanced), replication-type |
| kubernetes.io/azure-disk | Azure | storageaccounttype (Standard_LRS/Premium_LRS), kind (Shared/Dedicated) |
| kubernetes.io/cinder | OpenStack | availability, type |
| csi.driver.name | Generic CSI | Driver-specific parameters |
| nfs-client | NFS subdir | archiveOnDelete, pathPattern |

### Volume Binding Modes

**Immediate (Default):**
- Provisions PV immediately when PVC created
- May create volume in zone different from Pod requirements
- Can cause scheduling failures if volume and Pod zones mismatch

**WaitForFirstConsumer:**
- Delays provisioning until Pod using PVC is created and scheduled
- Ensures volume created in same zone as Pod
- Recommended for multi-zone clusters

```yaml
volumeBindingMode: WaitForFirstConsumer
```

### Default StorageClass

Mark one StorageClass as default for PVCs that don't specify one:

```yaml
metadata:
  annotations:
    storageclass.kubernetes.io/is-default-class: "true"
```

**Checking Default:**
```bash
kubectl get storageclass
# NAME                 PROVISIONER           RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
# fast-ssd (default)   kubernetes.io/gce-pd   Delete          WaitForFirstConsumer   true                  30d
# standard             kubernetes.io/gce-pd   Delete          Immediate              false                 30d
```

## 16.5 Dynamic Provisioning

Dynamic provisioning eliminates manual PV creation. When a PVC requests a StorageClass, the provisioner automatically creates the underlying storage asset and corresponding PV object.

### Provisioning Flow

1. User creates PVC referencing StorageClass "fast-ssd"
2. Kubernetes API server recognizes StorageClass provisioner (e.g., GCE PD CSI driver)
3. Provisioner creates cloud resource (GCP SSD disk)
4. Provisioner creates PV object with volume details
5. PVC binds to new PV
6. Pod mounts PVC

### CSI Drivers

Container Storage Interface (CSI) is the modern standard for storage plugins, replacing in-tree provisioners.

**Benefits:**
- Out-of-tree development (no core Kubernetes releases needed)
- Storage vendor maintains driver independently
- Advanced features (snapshots, cloning, expansion) via standardized API

**Common CSI Drivers:**
- AWS EBS CSI driver
- GCP Compute Persistent Disk CSI driver
- Azure Disk/File CSI driver
- NFS CSI driver
- Ceph CSI (RBD and CephFS)
- Portworx CSI
- VMware vSphere CSI

### Installation Example (AWS EBS CSI)

```bash
# Install CSI driver via Helm
helm repo add aws-ebs-csi-driver https://kubernetes-sigs.github.io/aws-ebs-csi-driver
helm install aws-ebs-csi-driver aws-ebs-csi-driver/aws-ebs-csi-driver \
  --namespace kube-system \
  --set enableVolumeSnapshot=true \
  --set controller.serviceAccount.annotations."eks\.amazonaws\.com/role-arn"=arn:aws:iam::ACCOUNT:role/EBS-CSI-Driver-Role

# Create StorageClass using CSI
cat <<EOF | kubectl apply -f -
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: ebs-sc
provisioner: ebs.csi.aws.com
volumeBindingMode: WaitForFirstConsumer
parameters:
  type: gp3
  encrypted: "true"
  kmsKeyId: alias/aws/ebs
  iops: "10000"
  throughput: "500"
EOF
```

### Volume Snapshots (CSI)

CSI enables point-in-time snapshots for backup and cloning:

```yaml
apiVersion: snapshot.storage.k8s.io/v1
kind: VolumeSnapshotClass
metadata:
  name: csi-snapclass
driver: ebs.csi.aws.com
deletionPolicy: Retain  # Keep snapshot when VolumeSnapshot deleted
parameters:
  tags: "key1=value1,key2=value2"
---
apiVersion: snapshot.storage.k8s.io/v1
kind: VolumeSnapshot
metadata:
  name: postgres-snapshot
  namespace: database
spec:
  volumeSnapshotClassName: csi-snapclass
  source:
    persistentVolumeClaimName: postgres-data
```

## 16.6 StatefulSets

StatefulSets manage deployment and scaling of stateful applications requiring:
- Stable, unique network identifiers
- Stable, persistent storage
- Ordered, graceful deployment and scaling
- Ordered, automated rolling updates

### StatefulSet vs Deployment

| Feature | Deployment | StatefulSet |
|---------|------------|-------------|
| Pod Naming | Random hash suffixes | Ordinal index (0, 1, 2...) |
| Storage | Shared PVC (ReadWriteMany) or ephemeral | Unique PVC per Pod (volumeClaimTemplates) |
| Ordering | Parallel creation/deletion | Sequential, ordered |
| Network | Random IPs | Stable DNS: `<pod-name>.<service-name>` |
| Scaling | Random deletion | Reverse ordinal termination |

### StatefulSet Specification

```yaml
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: postgres
  namespace: database
spec:
  serviceName: postgres-headless  # Headless service for stable network ID
  replicas: 3
  podManagementPolicy: OrderedReady  # Parallel alternative available
  selector:
    matchLabels:
      app: postgres
  template:
    metadata:
      labels:
        app: postgres
    spec:
      containers:
      - name: postgres
        image: postgres:15
        ports:
        - containerPort: 5432
          name: postgres
        env:
        - name: PGDATA
          value: /var/lib/postgresql/data/pgdata
        - name: POSTGRES_USER
          valueFrom:
            secretKeyRef:
              name: postgres-auth
              key: username
        - name: POSTGRES_PASSWORD
          valueFrom:
            secretKeyRef:
              name: postgres-auth
              key: password
        volumeMounts:
        - name: data
          mountPath: /var/lib/postgresql/data
        resources:
          requests:
            memory: "2Gi"
            cpu: "1000m"
          limits:
            memory: "4Gi"
            cpu: "2000m"
  volumeClaimTemplates:  # Creates PVC per replica
  - metadata:
      name: data
    spec:
      accessModes: ["ReadWriteOnce"]
      storageClassName: fast-ssd
      resources:
        requests:
          storage: 100Gi
```

**Generated Resources:**
- Pods: postgres-0, postgres-1, postgres-2
- PVCs: data-postgres-0, data-postgres-1, data-postgres-2
- DNS: postgres-0.postgres-headless.database.svc.cluster.local

### Headless Service Requirement

StatefulSets require a Headless Service for network identity:

```yaml
apiVersion: v1
kind: Service
metadata:
  name: postgres-headless
  namespace: database
spec:
  clusterIP: None  # Headless
  selector:
    app: postgres
  ports:
  - port: 5432
    name: postgres
```

**Discovery:**
Each Pod gets a stable DNS entry:
- `postgres-0.postgres-headless.database.svc.cluster.local`
- `postgres-1.postgres-headless.database.svc.cluster.local`

### Scaling Behavior

**Scaling Up:**
1. postgres-3 created and must be Running/Ready before postgres-4 created
2. PVC data-postgres-3 provisioned automatically
3. Pod mounts new PVC (empty if new, or existing data if scaled down and back up)

**Scaling Down:**
1. postgres-2 terminated first (highest ordinal)
2. PVC data-postgres-2 **not deleted** (data preserved)
3. If scaled back up, postgres-2 reclaims same PVC with previous data

**Manual Intervention:**
To delete PVC after scale-down:
```bash
kubectl delete pvc data-postgres-2
# Warning: Irreversible data loss
```

## 16.7 Database Persistence Strategies

Running databases in Kubernetes requires careful consideration of storage performance, backup, and high availability.

### Strategy 1: Single Pod with PV (Development)

Simplest approach suitable for development or non-critical workloads:

```yaml
apiVersion: apps/v1
kind: Deployment  # Note: Deployment, not StatefulSet
metadata:
  name: postgres-dev
spec:
  replicas: 1
  selector:
    matchLabels:
      app: postgres-dev
  template:
    spec:
      containers:
      - name: postgres
        image: postgres:15
        volumeMounts:
        - name: data
          mountPath: /var/lib/postgresql/data
      volumes:
      - name: data
        persistentVolumeClaim:
          claimName: postgres-dev-pvc
```

**Limitations:**
- No high availability (downtime during node maintenance)
- Potential data corruption if Pod crashes during write
- Manual failover required

### Strategy 2: StatefulSet with Replication (Production)

For PostgreSQL, use StatefulSet with streaming replication or Patroni for HA:

```yaml
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: postgres-primary
spec:
  serviceName: postgres
  replicas: 1  # Single writer; read replicas separate
  template:
    spec:
      containers:
      - name: postgres
        image: bitnami/postgresql-repmgr:15
        env:
        - name: POSTGRESQL_POSTGRES_PASSWORD
          valueFrom:
            secretKeyRef:
              name: postgres
              key: password
        - name: POSTGRESQL_USERNAME
          value: app_user
        - name: POSTGRESQL_PASSWORD
          valueFrom:
            secretKeyRef:
              name: postgres
              key: user-password
        - name: POSTGRESQL_DATABASE
          value: app_db
        - name: REPMGR_PRIMARY_HOST
          value: postgres-0.postgres
        - name: REPMGR_PRIMARY_PORT
          value: "5432"
        - name: REPMGR_PARTNER_NODES
          value: "postgres-0.postgres:5432,postgres-1.postgres:5432"
        volumeMounts:
        - name: data
          mountPath: /bitnami/postgresql
  volumeClaimTemplates:
  - metadata:
      name: data
    spec:
      accessModes: ["ReadWriteOnce"]
      storageClassName: fast-ssd
      resources:
        requests:
          storage: 500Gi
```

### Strategy 3: Cloud Managed Databases

For production, consider ExternalName Services pointing to managed databases:

```yaml
apiVersion: v1
kind: Service
metadata:
  name: postgres-production
  namespace: production
spec:
  type: ExternalName
  externalName: prod-db.abc123.us-east-1.rds.amazonaws.com
  ports:
  - port: 5432
```

**Benefits:**
- Automated backups and patching
- Multi-AZ replication
- Managed failover
- Performance insights

### Performance Optimization

**Storage Configuration:**
```yaml
volumeClaimTemplates:
- metadata:
    name: data
  spec:
    storageClassName: io2-block-storage  # AWS io2 for critical databases
    resources:
      requests:
        storage: 100Gi
    volumeMode: Block  # Raw block for databases
---
# Mount as block device in Pod
volumeMounts:
- name: data
  mountPath: /var/lib/mysql
volumes:
- name: data
  persistentVolumeClaim:
    claimName: data-mysql-0
```

**Database-Specific Settings:**
- PostgreSQL: Use `volumeMode: Filesystem` with `fsync` optimizations
- MySQL: Consider `volumeMode: Block` for InnoDB
- MongoDB: WiredTiger cache size tuning relative to memory limits

## 16.8 Backup and Restore

Data protection strategies for persistent volumes range from CSI snapshots to application-consistent backups.

### Method 1: CSI Snapshots

Leverage storage-level snapshots for point-in-time recovery:

```yaml
apiVersion: snapshot.storage.k8s.io/v1
kind: VolumeSnapshot
metadata:
  name: postgres-backup-$(date +%s)
  namespace: database
spec:
  volumeSnapshotClassName: csi-snapclass
  source:
    persistentVolumeClaimName: postgres-data
---
# Restore from snapshot
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: postgres-data-restored
spec:
  dataSource:
    name: postgres-backup-1234567890
    kind: VolumeSnapshot
    apiGroup: snapshot.storage.k8s.io
  accessModes:
    - ReadWriteOnce
  storageClassName: fast-ssd
  resources:
    requests:
      storage: 100Gi
```

**Limitations:**
- Crash-consistent (not application-consistent)
- Database may be mid-transaction
- Suitable for file systems, risky for active databases without coordination

### Method 2: Application-Level Backups

Use database-native tools for consistent backups:

**PostgreSQL with pg_dump:**
```yaml
apiVersion: batch/v1
kind: CronJob
metadata:
  name: postgres-backup
spec:
  schedule: "0 2 * * *"  # Daily at 2 AM
  concurrencyPolicy: Forbid
  jobTemplate:
    spec:
      template:
        spec:
          containers:
          - name: backup
            image: postgres:15
            command:
            - /bin/sh
            - -c
            - |
              pg_dump -h postgres -U admin app_db | \
              gzip > /backups/postgres-$(date +%Y%m%d-%H%M%S).sql.gz
            env:
            - name: PGPASSWORD
              valueFrom:
                secretKeyRef:
                  name: postgres-auth
                  key: password
            volumeMounts:
            - name: backups
              mountPath: /backups
          volumes:
          - name: backups
            persistentVolumeClaim:
              claimName: backup-storage
          restartPolicy: OnFailure
```

**Velero (Cluster Backup):**
Velero backs up Kubernetes resources and persistent volumes to cloud storage:

```bash
# Install Velero
velero install \
  --provider aws \
  --bucket my-backups \
  --secret-file ./credentials \
  --backup-location-config region=us-east-1 \
  --snapshot-location-config region=us-east-1 \
  --plugins velero/velero-plugin-for-aws:v1.7.0

# Create backup
velero backup create database-backup \
  --include-namespaces database \
  --include-resources persistentvolumeclaims,persistentvolumes,statefulsets,pods \
  --ttl 720h0m0s

# Restore
velero restore create --from-backup database-backup
```

### Backup Verification

Regularly test restore procedures:

```bash
# Restore to staging namespace
velero restore create --from-backup database-backup \
  --namespace-mappings database:database-restore-test

# Verify data integrity
kubectl exec -it -n database-restore-test postgres-0 -- psql -c "SELECT count(*) FROM critical_table;"
```

---

## Chapter Summary and Preview

In this chapter, we explored Kubernetes storage abstractions essential for stateful workloads. We distinguished ephemeral volumes (emptyDir, hostPath) suitable for temporary data from PersistentVolumes providing durable storage beyond Pod lifetimes. PersistentVolumeClaims enable developers to consume storage without infrastructure knowledge, while StorageClasses automate dynamic provisioning across cloud providers and on-premises storage. We examined StatefulSets as the primary workload controller for stateful applications, providing stable network identities and unique persistent storage per replica. Database persistence strategies ranged from simple single-Pod deployments for development to replicated StatefulSets with cloud-managed database integration for production. Finally, we established backup methodologies including CSI snapshots for crash-consistent recovery and application-level backups (pg_dump, Velero) for transaction-consistent data protection.

**Key Takeaways:**
- Always use PersistentVolumeClaims rather than directly referencing PersistentVolumes to maintain workload portability.
- Choose StorageClasses with `volumeBindingMode: WaitForFirstConsumer` for multi-zone clusters to ensure volume and Pod zone alignment.
- Use StatefulSets with volumeClaimTemplates for databases requiring stable storage identity; Deployments with PVCs suffice for single-instance caches.
- Never delete StatefulSet PVCs when scaling down unless explicitly intending data destruction.
- Implement application-consistent backups using native database tools or Velero rather than relying solely on CSI snapshots for critical data.

**Next Chapter Preview:**
Chapter 17: Advanced Kubernetes Concepts extends beyond basic workloads to explore DaemonSets for node-level services, Jobs and CronJobs for batch processing, Horizontal Pod Autoscaler (HPA) for dynamic scaling based on metrics, Vertical Pod Autoscaler (VPA) for right-sizing resources, Pod Disruption Budgets for graceful maintenance, and Resource Quotas for multi-tenant governance. These resources complete your operational toolkit for production Kubernetes management, enabling efficient resource utilization, automated scaling, and resilient batch processing alongside the stateful services established in this chapter.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='15. kubernetes_networking.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='17. advanced_kubernetes_concepts.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
